In [1]:
from keras_gym_env import ConnectFourEnv
import numpy as np
import datetime
import tensorflow as tf

from agent import DQNAgent
from buffer import Buffer
from training import train

#Subfolder for Logs
config_name = "default"
#createsummary writer for vusalization in tensorboard    
time_string = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# time_string = ""

best_train_path = f"logs/{config_name}/{time_string}/best_train"
adapting_train_path = f"logs/{config_name}/{time_string}/adapting_train"
best_train_writer = tf.summary.create_file_writer(best_train_path)
adapting_train_writer = tf.summary.create_file_writer(adapting_train_path)

best_test_path = f"logs/{config_name}/{time_string}/best_test"
adapting_test_path = f"logs/{config_name}/{time_string}/adapting_test"
best_test_writer = tf.summary.create_file_writer(best_test_path)
adapting_test_writer = tf.summary.create_file_writer(adapting_test_path)

train_writer = [best_train_writer, adapting_train_writer]
test_writer = [best_test_writer, adapting_test_writer]

model_path_best = f"model/{config_name}/{time_string}/best"
model_path_adapting = f"model/{config_name}/{time_string}/adapting"

# Hyperparameter
iterations = 12
BATCH_SIZE = 6
reward_function_adapting_agent = lambda d,r: tf.where(d, tf.where(r==0.0,tf.constant(1.0),tf.constant(0.0)), r)
EPSILON = 1 #TODO

# create buffer
best_buffer = Buffer(100000,1000)
adapting_buffer = Buffer(100000,1000)

# create agent
env = ConnectFourEnv()
best_agent = DQNAgent(env,best_buffer, batch = BATCH_SIZE, model_path = model_path_best)
adapting_agent = DQNAgent(env, adapting_buffer, batch = BATCH_SIZE, model_path = model_path_adapting, reward_function = reward_function_adapting_agent)
agents = [best_agent, adapting_agent]

train(agents, BATCH_SIZE, iterations, train_writer, test_writer, epsilon= EPSILON, epsilon_decay = 0.9)


print("done")


2023-03-09 12:39:50.642094: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 12:39:50.813100: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-09 12:39:51.308973: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fkirsch/anaconda3/envs/iannwtf/lib/
2023-03-09 12:39:51.309056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

























done
